

# Source :
https://python.plainenglish.io/connect-xero-oauth-2-0-using-python-without-a-website-86c79474c482



Notes : I also tested, you can create two apps with different client ids , they will work in on the same xero data


### format of Trial Balance
https://developer.xero.com/documentation/api/accounting/reports/#budget-summary

### XERO API Exploration
https://api-explorer.xero.com/accounting/reports/getreporttrialbalance?query-date=2022-06-30

### Auth flow - secret code
https://developer.xero.com/documentation/guides/oauth2/auth-flow#4-receive-your-tokens


### Auth flow - PKCE
https://developer.xero.com/documentation/guides/oauth2/pkce-flow

In [243]:
import pkce 
code_verifier = pkce.generate_code_verifier(length=128)
code_challenge = pkce.get_code_challenge(code_verifier)

In [244]:
#client_id = '9BFEB8611B8643FA846874C3E39512D7' # this is from xero developer web. click create new app
#client_id = '965AC403C2E0420384E1EF7FEBD1F117' # initial client id
#client_id = 'E1CA188B973D46B48AC8E74164B48A57' # second client id

client_id = '7FE3EB1A5A43422690AFF5C850F2B2DF'  # LCA Production New APP ID (Made by me)

redirect_url = 'https://developer.xero.com/'

In [245]:
#scope = 'offline_access accounting.contacts'
scope = 'offline_access accounting.reports.read'

Generate authentication URL and connect to xero

In [246]:
auth_url = ('https://login.xero.com/identity/connect/authorize?' +
'response_type=code' +
'&client_id=' + client_id +
'&redirect_uri=' + redirect_url +
'&scope=' + scope +
'&code_challenge=' + code_challenge +
'&code_challenge_method=S256' )

In [247]:
print (auth_url)

https://login.xero.com/identity/connect/authorize?response_type=code&client_id=7FE3EB1A5A43422690AFF5C850F2B2DF&redirect_uri=https://developer.xero.com/&scope=offline_access accounting.reports.read&code_challenge=Nkgon4kLJdm8rSCJT8YaGvRLtfqYUnDlEIgOzYoAs1g&code_challenge_method=S256


In [248]:
code = 'b0c6bb255dd7f3fb49b2e2b6aec8e1ae135ef94704d94e88d9736f4bf5f5508d'

Exchange token

In [249]:
import requests
exchange_code_url = 'https://identity.xero.com/connect/token'
response = requests.post(exchange_code_url,
           headers={
               'Content-Type': 'application/x-www-form-urlencoded'
           },
           data={
              'grant_type': 'authorization_code',
              'client_id' : client_id,
              'code': code,
              'redirect_uri': redirect_url,
              'code_verifier' : code_verifier
})

In [253]:
access_token = response.json()['access_token']
refresh_token = response.json()['refresh_token']

# save refresh token
f = open('LCAPKCE.txt','w')
f.write(refresh_token)

64

# Test to see how many tenants

In [254]:
url = 'https://api.xero.com/connections'
response = requests.get(url,
           headers={
               'Authorization' : 'Bearer '+access_token,
               'Accept' : 'application/json'})
json.loads(response.text)
  

[{'id': '31066cc7-6e47-4bf0-adba-1cb0a221f551',
  'authEventId': 'b569ccb3-c13d-4ff9-8301-41d77c108121',
  'tenantId': 'bd0b016e-409d-4784-becd-ba27c371757e',
  'tenantType': 'ORGANISATION',
  'tenantName': 'Laser Clinics Australia (Brookvale) Pty Limited',
  'createdDateUtc': '2022-05-17T07:17:55.2658500',
  'updatedDateUtc': '2022-05-17T07:17:55.2677210'},
 {'id': '69fbc816-5281-4aeb-9901-cdd5ae83a957',
  'authEventId': '216fbafc-a519-4a11-bddb-1226d721b745',
  'tenantId': 'ee698365-4b74-48b5-b453-d5d9c1551650',
  'tenantType': 'ORGANISATION',
  'tenantName': 'Laser Clinics Australia (Bondi) Pty Limited',
  'createdDateUtc': '2022-05-17T07:02:38.1573670',
  'updatedDateUtc': '2022-05-17T07:02:38.1586680'}]

In [232]:

# save to text file, try to call again later
newtoken = response.json()['refresh_token']


f = open('refreshtoken.txt','w') 
f.write(refresh_token)
f.close()
      

In [256]:

import pandas as pd

final = pd.DataFrame(columns=['Date','XeroOrganisation','SectionName','AccountCode','AccountName','Debit','Credit','YTD Debit','YTD Credit'])

############ get list of tenants ################
url = 'https://api.xero.com/connections'
response = requests.get(url,
           headers={
               'Authorization' : 'Bearer '+access_token,
               'Accept' : 'application/json'})
myjson = json.loads(response.text)

mytenants = []
for i in myjson:
    mytenants.append(i['tenantId'])
###################################################


############ put parameter for month ##############
########## Change URL for TRIAL BALANCE ###########

mymonth = '2022-05-31'
url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance?date='+mymonth+'&paymentsOnly=false'

###################################################


######### Get JSON API for each tenant ############
for mytenant in mytenants:
    response = requests.get(url,
            headers={
           'Authorization' : 'Bearer '+access_token,
           'Accept' : 'application/json',
           'Xero-tenant-id' : mytenant})
    data = json.loads(response.text)
###################################################



#### Modified the Json to DataFrame and CSV #######
    for i in data['Reports']:
        for j in i['ReportTitles']:
            if 'Trial' not in j and 'As at' not in j:
                clinicname = j

    # get Values from JSON API
    mycell = []
    for i in data['Reports']:
        for j, l1 in enumerate(i['Rows']):
            # only get the data from second row onwards, ignore headers 
            if j != 0:

                # grab section
                mysection = l1['Title']

                # go to dictionary rows section and loop it to find cells
                # from cells you can get all the values for each row
                for l2 in l1['Rows']:
                    #print (l2,'\n')
                    myrow = []
                    myrow.append(mysection)
                    for l3 in l2['Cells']:
                      #print (l3['Value'])
                      myrow.append(l3['Value'])
                    mycell.append(myrow)

    # create draft dataframe                
    df = pd.DataFrame(mycell)
    df.columns = ['Section','Desc','Dr','Cr','YTD Dr','YTD Cr']

    # add additional information to dataframe
    df['ClinicName'] = clinicname
    df['Month'] = mymonth
    df['AccountCode'] = df['Desc'].apply(lambda x:x[-4:-1])
    df['Desc'] = df['Desc'].apply(lambda x:x[:-6])

    # final dataframe
    df = df[['Month','ClinicName','Section','AccountCode','Desc','Dr','Cr','YTD Dr','YTD Cr']]
    df.columns = ['Date','XeroOrganisation','SectionName','AccountCode','AccountName','Debit','Credit','YTD Debit','YTD Credit']

    df.to_csv(clinicname+'.csv',index=False)
    final = final.append(df)
######################################################   

In [257]:
final

,Date,XeroOrganisation,SectionName,AccountCode,AccountName,Debit,Credit,YTD Debit,YTD Credit
0,2022-05-31,Laser Clinics Australia (Brookvale) Pty Limited,Revenue,000,Earned Revenue (,,74755.75,,1801494.84
1,2022-05-31,Laser Clinics Australia (Brookvale) Pty Limited,Revenue,000,Interest Income (,,2.14,,20.31
2,2022-05-31,Laser Clinics Australia (Brookvale) Pty Limited,Revenue,000,Recoveries - Clinics (,,0.00,,75.91
3,2022-05-31,Laser Clinics Australia (Brookvale) Pty Limited,Revenue,010,Sales Commission (,,0.00,,477.00
4,2022-05-31,Laser Clinics Australia (Brookvale) Pty Limited,Expenses,500,Opening Stock (,60933.25,,581189.57,
...,...,...,...,...,...,...,...,...,...
88,2022-05-31,Laser Clinics Australia (Bondi) Pty Limited,Equity,113,Tax Paid FY13 (,,0.00,23454.90,
89,2022-05-31,Laser Clinics Australia (Bondi) Pty Limited,Equity,114,Tax Paid FY14 (,,0.00,41016.30,
90,2022-05-31,Laser Clinics Australia (Bondi) Pty Limited,Equity,115,Tax Paid FY15 (,,0.00,126908.70,
91,2022-05-31,Laser Clinics Australia (Bondi) Pty Limited,Equity,116,Tax Paid FY16 (,,0.00,33273.30,
